In [300]:
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
%matplotlib inline

In [180]:
fx_list = ["aud", "cad", "chf", "eur", "gbp", "jpy", "nok", "nzd", "sek"]

Note that we choose time from 2013-10-01 to 018-09-30.

In [203]:
data_path = "./g10/"
def get_data(price_type):
    convert = {"price":"PX_LAST", "ask":"PX_ASK", "bid":"PX_BID"}
    label = convert[price_type]
    for i in range(len(fx_list)):
        if i == 0:
            data = pd.read_excel(data_path + fx_list[i] + price_type + ".xlsx", header=6)
            data.rename(columns={label:fx_list[i]+"_"+price_type}, inplace=True)
        else:
            data[fx_list[i]+"_"+price_type] = pd.read_excel(data_path + fx_list[i] + price_type + ".xlsx", header=6)[label]

    period = (data["Date"] >= '2013-10-01') & (data["Date"] <= '2018-09-30')
    data = data.loc[period]    
    data.set_index(pd.DatetimeIndex(data["Date"]), inplace=True)
    del data["Date"]
    
    return data

In [340]:
market_data = get_data(price_type="price")[::-1]
market_data.head()

,aud_price,cad_price,chf_price,eur_price,gbp_price,jpy_price,nok_price,nzd_price,sek_price
Date,,,,,,,,,
2013-10-01,0.9398,0.9687,1.1044,1.3526,1.6196,0.010204,0.1668,0.8276,0.157220
2013-10-02,0.9385,0.9674,1.1077,1.3579,1.6227,0.010272,0.1664,0.8330,0.156753
2013-10-03,0.9394,0.9678,1.1118,1.3619,1.6156,0.010282,0.1679,0.8293,0.157584
2013-10-04,0.9435,0.9713,1.1023,1.3558,1.6011,0.010259,0.1675,0.8317,0.155569
2013-10-07,0.9429,0.9697,1.1076,1.3581,1.6097,0.010340,0.1675,0.8305,0.155908


In [341]:
for fx in fx_list:
    market_data[fx+"_ret"] = -market_data[fx+"_price"].shift(1)/market_data[fx+'_price'] + 1
    del market_data[fx+"_price"]

market_data.head()

,aud_ret,cad_ret,chf_ret,eur_ret,gbp_ret,jpy_ret,nok_ret,nzd_ret,sek_ret
Date,,,,,,,,,
2013-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-02,-0.001385,-0.001344,0.002979,0.003903,0.001910,0.006620,-0.002404,0.006483,-0.002979
2013-10-03,0.000958,0.000413,0.003688,0.002937,-0.004395,0.000973,0.008934,-0.004462,0.005273
2013-10-04,0.004346,0.003603,-0.008618,-0.004499,-0.009056,-0.002242,-0.002388,0.002886,-0.012952
2013-10-07,-0.000636,-0.001650,0.004785,0.001694,0.005343,0.007834,0.000000,-0.001445,0.002174


## Signal:

### RSI (Relative Strength Index)

* Category: Momentum
* Calculation: Assume there are `x` days of rise and `n-x` days of in the previous `n` days. Get the average rise in `x` days as `Xr`, and average drop as `Xd`, then `RSI = Xr / (Xd + Xr) * 100`. A recommended default value for `n` is 14.
* Result: should be in `[0, 100]`. If `>50` it indicates a long signal.

In [184]:
def RSI(rets):
    # input rets is a list of returns
    rise, drop = 0, 0
    rise_num, drop_num = 0, 0
    for ret in rets:
        if ret > 0:
            rise += ret
            rise_num += 1
        elif ret < 0:
            drop -= ret
            drop_num += 1
    avg_rise = rise / rise_num if rise_num > 0 else 0
    avg_drop = drop / drop_num if drop_num > 0 else 0
    if (avg_rise + avg_drop) == 0:
        print("avg rise and drop all zero:", rets)
        return 50
    return avg_rise / (avg_rise + avg_drop) * 100

In [185]:
def transform_RSI(data):
    for col in data.columns:
        new_col = col[:4]+"rsi"
        data[new_col] = 0
        # start from 15th day, 1 day is NaN. Calculate first RSI at the end of 15th day.
        for i in range(15, data.shape[0]):
            data.ix[i, new_col] = RSI(list(data.iloc[i-14:i][col]))
            
    return data

In [342]:
%%time
data_rsi = transform_RSI(market_data)

D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


Wall time: 5.73 s


In [343]:
rsi_cols = data_rsi.columns[-9:]
data_rsi = data_rsi[rsi_cols]
data_rsi

,aud_rsi,cad_rsi,chf_rsi,eur_rsi,gbp_rsi,jpy_rsi,nok_rsi,nzd_rsi,sek_rsi
Date,,,,,,,,,
2013-10-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-10-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


###  Trading time frame
The earliest starting date of the trading can only be 2013-10-22. We set it as the first day of backtesting, and we test the strategy with different backtesting rebalancing frequency.

### Strategy implementation
We assume we have 1 million USD. At each rebalancing day, we will equally divide our money into 1/6, and long top 3 RSI fx and short last 3 RSI fx. If we long, we buy at ask price and calculate daily return based on bid price.

In [344]:
start_date = 15  # "2013-10-22"
freq = 15

def fx_select(date_index,data_rsi):
    rsi_dict = dict(zip(rsi_cols, data_rsi.iloc[date_index]))
    long_fx = [i[:3] for i in sorted(rsi_dict, key=rsi_dict.get)[-3:]]
    short_fx = [i[:3] for i in sorted(rsi_dict, key=rsi_dict.get)[:3]]
    return long_fx, short_fx

In [345]:
fx_select(start_date, data_rsi)

(['cad', 'chf', 'aud'], ['sek', 'gbp', 'jpy'])

In [310]:
# load bid price and ask price
bid_data = get_data(price_type="bid")[::-1]
ask_data = get_data(price_type="ask")[::-1]

In [346]:
def asset_alloc(data, freq, start_date=start_date):
    cols = data.columns
    for col in cols:
        data[col[:3]] = 0
    for i in range(start_date, data.shape[0]):
        if (i-start_date) % freq == 0:
            long_fx, short_fx = fx_select(i, data)
            for long in long_fx:
                data.ix[i, long] = 1/6
            for short in short_fx:
                data.ix[i, short] = -1/6 # indicate short
        else:
            data.ix[i] = data.ix[i-1]
                
    return data[data.columns[-9:]]


In [347]:
data_alloc = asset_alloc(data_rsi, freq=freq).iloc[start_date:]

D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: Dep

In [348]:
data_alloc

,aud,cad,chf,eur,gbp,jpy,nok,nzd,sek
Date,,,,,,,,,
2013-10-22,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-23,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-24,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-25,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-28,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-29,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-30,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-10-31,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667
2013-11-01,0.166667,0.166667,0.166667,0.000000,-0.166667,-0.166667,0.000000,0.000000,-0.166667


In [313]:
def backtest(freq, ask=ask_data[start_date:], bid=bid_data[start_date:], alloc=data_alloc, tranx_cost=0):
    fx_list = ['aud', 'cad', 'chf', 'eur', 'gbp', 'jpy', 'nok', 'nzd', 'sek']
    alloc["return"] = 0
    alloc.ix[0, "return"] = 1
    for i in range(1, alloc.shape[0]): # the return we get starts from 2nd day
        ret = 0
        if i % freq == 1:
            value = alloc.iloc[i-1]["return"]
            for fx in fx_list:
                percent = alloc.iloc[i-1][fx]
                if percent > 0:
                    ret += bid.iloc[i][fx+"_bid"] / ask.iloc[i-1][fx+"_ask"] * value * percent * (1-tranx_cost)
                elif percent < 0:
                    ret -= bid.iloc[i-1][fx+"_bid"] / ask.iloc[i][fx+"_ask"] * value * percent * (1-tranx_cost)
            alloc.ix[i, "return"] = ret
        else:
            for fx in fx_list:
                percent = alloc.iloc[i-1][fx]
                # here ret is the proportional return, will muptily the last time return
                if percent > 0:
                    ret += bid.iloc[i][fx+"_bid"] / bid.iloc[i-1][fx+"_bid"]
                elif percent < 0:
                    ret += ask.iloc[i-1][fx+"_ask"] / ask.iloc[i][fx+"_ask"]
            alloc.ix[i, "return"] = alloc.ix[i-1, "return"] * ret / 6
    
    return alloc["return"]

In [349]:
data_ret = backtest(freq=freq)

D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
D:\Programming\Anaconda3\lib\site-packages\ipykernel\__main__.py:24: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [352]:
data_ret

Date
2013-10-22    1.000000
2013-10-23    0.996755
2013-10-24    0.995084
2013-10-25    0.993390
2013-10-28    0.994277
2013-10-29    0.994570
2013-10-30    0.995237
2013-10-31    0.996236
2013-11-01    0.998197
2013-11-04    0.998647
2013-11-05    0.996806
2013-11-06    0.997157
2013-11-07    0.994377
2013-11-08    0.995606
2013-11-11    0.995364
2013-11-12    0.998486
2013-11-13    0.997995
2013-11-14    0.998030
2013-11-15    0.998989
2013-11-18    0.999021
2013-11-19    0.999739
2013-11-20    1.000619
2013-11-21    0.997937
2013-11-22    0.994710
2013-11-25    0.995448
2013-11-26    0.995007
2013-11-27    0.994386
2013-11-28    0.992753
2013-11-29    0.993342
2013-12-02    0.995487
                ...   
2018-08-20    0.968061
2018-08-21    0.968723
2018-08-22    0.965456
2018-08-23    0.965142
2018-08-24    0.965246
2018-08-27    0.965875
2018-08-28    0.967395
2018-08-29    0.964909
2018-08-30    0.964853
2018-08-31    0.964874
2018-09-03    0.965400
2018-09-04    0.963893
2018-0